#Automação: Criação de Gráficos e Relatórios

In [ ]:
!pip install fpdf

import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import re
import io
import os
from google.colab import files

In [ ]:
def processar_vários_relatorios(arquivo_csv, caminho_drive='seu_repositorio_drive'):

    try:
        df = pd.read_csv(arquivo_csv, sep=';', encoding='utf-8-sig')
    except:
        df = pd.read_csv(arquivo_csv, sep=';', encoding='latin1')

    df = df.dropna(subset=['Cliente'])
    df['Cliente'] = df['Cliente'].astype(str).str.strip()
    clientes_unicos = [c for c in df['Cliente'].unique() if c.strip() != '']

    if not os.path.exists(caminho_drive):
        print(f"⚠️ Drive não encontrado em {caminho_drive}. Usando pasta local 'Relatorios_Temporarios'.")
        caminho_saida = '/content/Relatorios_Temporarios'
        os.makedirs(caminho_saida, exist_ok=True)
        fazer_download_ao_final = True
    else:
        caminho_saida = caminho_drive
        fazer_download_ao_final = False

    for nome_cliente in clientes_unicos:
        dados_cliente = df[df['Cliente'] == nome_cliente]
        nome_arquivo_seguro = re.sub(r'[.\\/*?:"<>|]', "", nome_cliente).replace(" ", "_")

        plt.figure(figsize=(7, 4))
        plt.plot(dados_cliente['Meses'], dados_cliente['Eficiencia (%)'], marker='o', color='teal', linewidth=2)
        plt.title(f'Performance Operacional - {nome_cliente}')
        plt.xlabel('Mês')
        plt.ylabel('Eficiência (%)')
        plt.xticks(rotation=45)
        plt.grid(True, linestyle='--', alpha=0.7)

        # Salvando em um buffer de memória
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        plt.close()
        buffer.seek(0) # Volta ao início do "arquivo virtual"

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", 'B', 16)
        pdf.cell(200, 15, txt=f"RELATÓRIO TÉCNICO: {nome_cliente}", ln=True, align='C')

        pdf.set_font("Arial", size=11)
        pdf.ln(5)
        pdf.multi_cell(0, 7, txt=f"O presente documento apresenta os indicadores de eficiência da unidade {nome_cliente}.")

        with open("temp_img.png", "wb") as f:
            f.write(buffer.getbuffer())

        pdf.ln(10)
        pdf.image("temp_img.png", x=15, w=180)
        os.remove("temp_img.png") # Remove a imagem imediatamente após inserir no PDF

        caminho_final_pdf = os.path.join(caminho_saida, f"Relatorio_{nome_arquivo_seguro}.pdf")
        pdf.output(caminho_final_pdf)
        print(f"✅ Gerado: {caminho_final_pdf}")

    if fazer_download_ao_final:
        print("📦 Compactando e baixando relatórios...")
        os.system(f"zip -r relatorios.zip {caminho_saida}")
        files.download("relatorios.zip")

processar_vários_relatorios('/content/empresas_dados_automacao.csv')